# Summary

This file writes csv files into sqlite3 database

# Using sql

## write csvs to sql

## toy example (illustation)

In [24]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('test_database')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS products (product_name text, price number)')
conn.commit()

data = {'product_name': ['Computer','Tablet','Monitor','Printer'],
        'price': [900,300,450,150]
        }

df = pd.DataFrame(data, columns= ['product_name','price'])
df.to_sql('products', conn, if_exists='replace', index = False)


c.execute('''  
SELECT * FROM products
          ''')

for row in c.fetchall():
    print (row)

('Computer', 900)
('Tablet', 300)
('Monitor', 450)
('Printer', 150)


## connect to database

In [4]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(database='aqf_database')
c = conn.cursor()

### read from csv and write to sql

In [5]:
sp500_op_ret = pd.read_csv("data/sp500_op_ret.csv")

In [6]:
c.execute(f'CREATE TABLE IF NOT EXISTS sp500_op_ret ({", ".join(list(sp500_op_ret.columns))})')
conn.commit()

sp500_op_ret.to_sql('sp500_op_ret', conn, if_exists='replace', 
                    index = False, chunksize=10000)

In [16]:
mapping_table = pd.read_csv("data/mapping_table.csv")

In [17]:
c.execute(f'CREATE TABLE IF NOT EXISTS mapping_table ({", ".join(list(mapping_table.columns))})')
conn.commit()

mapping_table.to_sql('mapping_table', conn, if_exists='replace', 
                    index = False, chunksize=10000)

## write in chunks 
(roughly 12 mins)

In [9]:
signed_predictors_dl_wide = pd.read_csv("data/signed_predictors_dl_wide.csv")

In [10]:
c.execute(f'CREATE TABLE IF NOT EXISTS signed_predictors_dl_wide ({", ".join(list(signed_predictors_dl_wide.columns))})')
conn.commit()

signed_predictors_dl_wide.to_sql('signed_predictors_dl_wide', conn, if_exists='replace', 
                                index = False, chunksize=10000)

## Retrieve data from database

In [33]:
c.execute('''DROP TABLE sp500_op_ret_permno''')

query = c.execute(''' 
        CREATE TABLE sp500_op_ret_permno
        AS 
            SELECT 
                op.secid as secid, op.date as date, op.exdate as exdate, op.cp_flag as cp_flag, op.strike_price as strike_price, op.best_bid as best_bid, op.best_offer as best_offer, op.volume as volume, op.open_interest as open_interest, op.impl_volatility as impl_volatility, op.delta as delta, op.gamma as gamma, op.vega as vega, op.theta as theta, op.optionid as optionid, op.cfadj as cfadj, op.days_no_trading as days_no_trading, op.days_to_exp as days_to_exp, op.forwardprice as forwardprice, op.spotprice as spotprice, op.adj_spot as adj_spot, op.ir_rate as ir_rate, op.mid_price as mid_price, op.option_ret as option_ret,
                mapping_table.permno as permno
            FROM sp500_op_ret AS op
            LEFT JOIN mapping_table 
            ON op.secid = mapping_table.secid
          ''')

In [6]:
query = c.execute('''SELECT * from signed_predictors_dl_wide''')
", ".join(list(map(lambda x: f"pred.{x[0]} as {x[0]}" , query.description)))

'pred.permno as permno, pred.yyyymm as yyyymm, pred.AbnormalAccruals as AbnormalAccruals, pred.Accruals as Accruals, pred.AccrualsBM as AccrualsBM, pred.Activism1 as Activism1, pred.Activism2 as Activism2, pred.AdExp as AdExp, pred.AgeIPO as AgeIPO, pred.AM as AM, pred.AnalystRevision as AnalystRevision, pred.AnalystValue as AnalystValue, pred.AnnouncementReturn as AnnouncementReturn, pred.AOP as AOP, pred.AssetGrowth as AssetGrowth, pred.Beta as Beta, pred.BetaFP as BetaFP, pred.BetaLiquidityPS as BetaLiquidityPS, pred.BetaTailRisk as BetaTailRisk, pred.betaVIX as betaVIX, pred.BidAskSpread as BidAskSpread, pred.BM as BM, pred.BMdec as BMdec, pred.BookLeverage as BookLeverage, pred.BPEBM as BPEBM, pred.BrandInvest as BrandInvest, pred.Cash as Cash, pred.CashProd as CashProd, pred.CBOperProf as CBOperProf, pred.CF as CF, pred.cfp as cfp, pred.ChangeInRecommendation as ChangeInRecommendation, pred.ChAssetTurnover as ChAssetTurnover, pred.ChEQ as ChEQ, pred.ChForecastAccrual as ChForecas

In [9]:
query = c.execute('''
    CREATE TABLE sp500_op_ret_with_all_features
    AS
        SELECT 
            op.secid as secid, op.date as date, op.exdate as exdate, op.cp_flag as cp_flag, op.strike_price as strike_price, op.best_bid as best_bid, op.best_offer as best_offer, op.volume as volume, op.open_interest as open_interest, op.impl_volatility as impl_volatility, op.delta as delta, op.gamma as gamma, op.vega as vega, op.theta as theta, op.optionid as optionid, op.cfadj as cfadj, op.days_no_trading as days_no_trading, op.days_to_exp as days_to_exp, op.forwardprice as forwardprice, op.spotprice as spotprice, op.adj_spot as adj_spot, op.ir_rate as ir_rate, op.mid_price as mid_price, op.option_ret as option_ret, op.permno as permno,
            pred.permno as permno, pred.yyyymm as yyyymm, pred.AbnormalAccruals as AbnormalAccruals, pred.Accruals as Accruals, pred.AccrualsBM as AccrualsBM, pred.Activism1 as Activism1, pred.Activism2 as Activism2, pred.AdExp as AdExp, pred.AgeIPO as AgeIPO, pred.AM as AM, pred.AnalystRevision as AnalystRevision, pred.AnalystValue as AnalystValue, pred.AnnouncementReturn as AnnouncementReturn, pred.AOP as AOP, pred.AssetGrowth as AssetGrowth, pred.Beta as Beta, pred.BetaFP as BetaFP, pred.BetaLiquidityPS as BetaLiquidityPS, pred.BetaTailRisk as BetaTailRisk, pred.betaVIX as betaVIX, pred.BidAskSpread as BidAskSpread, pred.BM as BM, pred.BMdec as BMdec, pred.BookLeverage as BookLeverage, pred.BPEBM as BPEBM, pred.BrandInvest as BrandInvest, pred.Cash as Cash, pred.CashProd as CashProd, pred.CBOperProf as CBOperProf, pred.CF as CF, pred.cfp as cfp, pred.ChangeInRecommendation as ChangeInRecommendation, pred.ChAssetTurnover as ChAssetTurnover, pred.ChEQ as ChEQ, pred.ChForecastAccrual as ChForecastAccrual, pred.ChInv as ChInv, pred.ChInvIA as ChInvIA, pred.ChNAnalyst as ChNAnalyst, pred.ChNNCOA as ChNNCOA, pred.ChNWC as ChNWC, pred.ChTax as ChTax, pred.CitationsRD as CitationsRD, pred.CompEquIss as CompEquIss, pred.CompositeDebtIssuance as CompositeDebtIssuance, pred.ConsRecomm as ConsRecomm, pred.ConvDebt as ConvDebt, pred.CoskewACX as CoskewACX, pred.Coskewness as Coskewness, pred.CredRatDG as CredRatDG, pred.CustomerMomentum as CustomerMomentum, pred.DebtIssuance as DebtIssuance, pred.DelBreadth as DelBreadth, pred.DelCOA as DelCOA, pred.DelCOL as DelCOL, pred.DelDRC as DelDRC, pred.DelEqu as DelEqu, pred.DelFINL as DelFINL, pred.DelLTI as DelLTI, pred.DelNetFin as DelNetFin, pred.DivInit as DivInit, pred.DivOmit as DivOmit, pred.DivSeason as DivSeason, pred.DivYieldST as DivYieldST, pred.dNoa as dNoa, pred.DolVol as DolVol, pred.DownRecomm as DownRecomm, pred.EarningsConsistency as EarningsConsistency, pred.EarningsForecastDisparity as EarningsForecastDisparity, pred.EarningsStreak as EarningsStreak, pred.EarningsSurprise as EarningsSurprise, pred.EarnSupBig as EarnSupBig, pred.EBM as EBM, pred.EntMult as EntMult, pred.EP as EP, pred.EquityDuration as EquityDuration, pred.ExchSwitch as ExchSwitch, pred.ExclExp as ExclExp, pred.FEPS as FEPS, pred.fgr5yrLag as fgr5yrLag, pred.FirmAge as FirmAge, pred.FirmAgeMom as FirmAgeMom, pred.ForecastDispersion as ForecastDispersion, pred.FR as FR, pred.Frontier as Frontier, pred.Governance as Governance, pred.GP as GP, pred.GrAdExp as GrAdExp, pred.grcapx as grcapx, pred.grcapx3y as grcapx3y, pred.GrLTNOA as GrLTNOA, pred.GrSaleToGrInv as GrSaleToGrInv, pred.GrSaleToGrOverhead as GrSaleToGrOverhead, pred.Herf as Herf, pred.HerfAsset as HerfAsset, pred.HerfBE as HerfBE, pred.High52 as High52, pred.hire as hire, pred.IdioRisk as IdioRisk, pred.IdioVol3F as IdioVol3F, pred.IdioVolAHT as IdioVolAHT, pred.Illiquidity as Illiquidity, pred.IndIPO as IndIPO, pred.IndMom as IndMom, pred.IndRetBig as IndRetBig, pred.IntanBM as IntanBM, pred.IntanCFP as IntanCFP, pred.IntanEP as IntanEP, pred.IntanSP as IntanSP, pred.IntMom as IntMom, pred.Investment as Investment, pred.InvestPPEInv as InvestPPEInv, pred.InvGrowth as InvGrowth, pred.IO_ShortInterest as IO_ShortInterest, pred.iomom_cust as iomom_cust, pred.iomom_supp as iomom_supp, pred.Leverage as Leverage, pred.LRreversal as LRreversal, pred.MaxRet as MaxRet, pred.MeanRankRevGrowth as MeanRankRevGrowth, pred.Mom12m as Mom12m, pred.Mom12mOffSeason as Mom12mOffSeason, pred.Mom6m as Mom6m, pred.Mom6mJunk as Mom6mJunk, pred.MomOffSeason as MomOffSeason, pred.MomOffSeason06YrPlus as MomOffSeason06YrPlus, pred.MomOffSeason11YrPlus as MomOffSeason11YrPlus, pred.MomOffSeason16YrPlus as MomOffSeason16YrPlus, pred.MomRev as MomRev, pred.MomSeason as MomSeason, pred.MomSeason06YrPlus as MomSeason06YrPlus, pred.MomSeason11YrPlus as MomSeason11YrPlus, pred.MomSeason16YrPlus as MomSeason16YrPlus, pred.MomSeasonShort as MomSeasonShort, pred.MomVol as MomVol, pred.MRreversal as MRreversal, pred.MS as MS, pred.NetDebtFinance as NetDebtFinance, pred.NetDebtPrice as NetDebtPrice, pred.NetEquityFinance as NetEquityFinance, pred.NetPayoutYield as NetPayoutYield, pred.NOA as NOA, pred.NumEarnIncrease as NumEarnIncrease, pred.OperProf as OperProf, pred.OperProfRD as OperProfRD, pred.OPLeverage as OPLeverage, pred.OptionVolume1 as OptionVolume1, pred.OptionVolume2 as OptionVolume2, pred.OrderBacklog as OrderBacklog, pred.OrderBacklogChg as OrderBacklogChg, pred.OrgCap as OrgCap, pred.OScore as OScore, pred.PatentsRD as PatentsRD, pred.PayoutYield as PayoutYield, pred.PctAcc as PctAcc, pred.PctTotAcc as PctTotAcc, pred.PredictedFE as PredictedFE, pred.PriceDelayRsq as PriceDelayRsq, pred.PriceDelaySlope as PriceDelaySlope, pred.PriceDelayTstat as PriceDelayTstat, pred.ProbInformedTrading as ProbInformedTrading, pred.PS as PS, pred.RD as RD, pred.RDAbility as RDAbility, pred.RDcap as RDcap, pred.RDIPO as RDIPO, pred.RDS as RDS, pred.realestate as realestate, pred.ResidualMomentum as ResidualMomentum, pred.retConglomerate as retConglomerate, pred.ReturnSkew as ReturnSkew, pred.ReturnSkew3F as ReturnSkew3F, pred.REV6 as REV6, pred.RevenueSurprise as RevenueSurprise, pred.RIO_Disp as RIO_Disp, pred.RIO_MB as RIO_MB, pred.RIO_Turnover as RIO_Turnover, pred.RIO_Volatility as RIO_Volatility, pred.roaq as roaq, pred.RoE as RoE, pred.sfe as sfe, pred.ShareIss1Y as ShareIss1Y, pred.ShareIss5Y as ShareIss5Y, pred.ShareRepurchase as ShareRepurchase, pred.ShareVol as ShareVol, pred.ShortInterest as ShortInterest, pred.sinAlgo as sinAlgo, pred.skew1 as skew1, pred.SmileSlope as SmileSlope, pred.SP as SP, pred.Spinoff as Spinoff, pred.std_turn as std_turn, pred.SurpriseRD as SurpriseRD, pred.tang as tang, pred.Tax as Tax, pred.TotalAccruals as TotalAccruals, pred.UpRecomm as UpRecomm, pred.VarCF as VarCF, pred.VolMkt as VolMkt, pred.VolSD as VolSD, pred.VolumeTrend as VolumeTrend, pred.XFIN as XFIN, pred.zerotrade as zerotrade, pred.zerotradeAlt1 as zerotradeAlt1, pred.zerotradeAlt12 as zerotradeAlt12
        FROM sp500_op_ret_permno AS op
        LEFT JOIN signed_predictors_dl_wide as pred
        ON op.permno = pred.permno
''')

OperationalError: database or disk is full

In [4]:
sql = '''
    SELECT op.secid
    FROM sp500_op_ret_permno AS op
    LEFT JOIN signed_predictors_dl_wide as pred
    ON op.permno = pred.permno
'''
cnt = 0
for chunk in pd.read_sql_query(sql , conn, chunksize=5):
    print(chunk)
    cnt += 1
    if cnt > 5:
        break

   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015
   secid
0   5015
1   5015
2   5015
3   5015
4   5015


In [ ]:
pd.DataFrame.from_records(data=query.fetchall())

In [30]:
query.description

In [ ]:
c.execute('''
SELECT name from sqlite_master where type= "table"
''')
c.fetchall()

In [31]:
c.execute('''
SELECT * from sp500_op_ret
''')
", ".join(
    list(map(lambda x: x[0], c.description))
)

'secid, date, exdate, cp_flag, strike_price, best_bid, best_offer, volume, open_interest, impl_volatility, delta, gamma, vega, theta, optionid, cfadj, days_no_trading, days_to_exp, forwardprice, spotprice, adj_spot, ir_rate, mid_price, option_ret'

In [13]:
df

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,...,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret
0,5015,1996-01-31 00:00:00,1996-07-20 00:00:00,C,20.0,4.625,5.000,15,25,0.380062,...,10980421,1,0,171,24.142672,24.000,24.0000,0.052253,4.8125,-0.025367
1,5048,1996-01-31 00:00:00,1996-08-17 00:00:00,P,40.0,3.000,3.375,5,37,0.245716,...,11613022,1,0,199,39.344930,39.000,39.0000,0.051780,3.1875,0.002347
2,5049,1996-01-31 00:00:00,1996-05-18 00:00:00,C,65.0,5.875,6.375,4,420,0.167298,...,11618802,1,0,108,70.168661,70.375,35.1875,0.053274,6.1250,-0.009721
3,5049,1996-01-31 00:00:00,1996-08-17 00:00:00,C,70.0,3.250,3.625,13,3806,0.149214,...,11770703,1,0,199,70.393348,70.375,35.1875,0.051780,3.4375,0.001186
4,5061,1996-01-31 00:00:00,1996-06-22 00:00:00,C,30.0,3.375,3.750,5,254,0.324523,...,10914516,1,0,143,32.001209,31.750,31.7500,0.052726,3.5625,-0.006649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825526,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,C,13.0,1.150,1.500,134,665,0.437066,...,143779767,1,0,227,12.112223,12.310,12.3100,0.002822,1.3250,-0.068384
3825527,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,C,15.0,0.100,0.750,5,14,0.335686,...,143779769,1,0,227,12.112223,12.310,12.3100,0.002822,0.4250,0.055073
3825528,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,P,11.0,0.200,1.100,3,111,0.308935,...,143779782,1,0,227,12.112223,12.310,12.3100,0.002822,0.6500,0.020419
3825529,214905,2021-11-30 00:00:00,2022-07-15 00:00:00,P,12.0,1.100,1.600,53,35,0.372075,...,143779783,1,0,227,12.112223,12.310,12.3100,0.002822,1.3500,-0.016824


# Use MongoDB (deprecated)

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
data = pd.read_csv("data/signed_predictors_dl_wide.csv") # nrows=100000)

In [5]:
data.shape

(4890049, 204)

In [51]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["aqf_database"]
mycol = mydb["signed_predictors_dl_wide"]

destroy existed collection

In [53]:
mycol.delete_many({})

insert 10000 rows costs around 3 seconds



so it takes around 500x3 = 1500 seconds, or around half an hour

In [54]:
chunk_size= 10000
for i in range(data.shape[0]//chunk_size + 1):
    if i % 10 == 0:
        print(f"...Have written {i * chunk_size} rows...")
    # takes around 3 seconds for per insert 
    mycol.insert_many(
        data.iloc[i*chunk_size:(i+1) * chunk_size].to_dict("records")
    ) 
    

...Have written 0 rows...
...Have written 100000 rows...
...Have written 200000 rows...
...Have written 300000 rows...
...Have written 400000 rows...
...Have written 500000 rows...
...Have written 600000 rows...
...Have written 700000 rows...
...Have written 800000 rows...
...Have written 900000 rows...
...Have written 1000000 rows...
...Have written 1100000 rows...
...Have written 1200000 rows...
...Have written 1300000 rows...
...Have written 1400000 rows...
...Have written 1500000 rows...
...Have written 1600000 rows...
...Have written 1700000 rows...
...Have written 1800000 rows...
...Have written 1900000 rows...
...Have written 2000000 rows...
...Have written 2100000 rows...
...Have written 2200000 rows...
...Have written 2300000 rows...
...Have written 2400000 rows...
...Have written 2500000 rows...
...Have written 2600000 rows...
...Have written 2700000 rows...
...Have written 2800000 rows...
...Have written 2900000 rows...
...Have written 3000000 rows...
...Have written 3100000